## In this assignment, I have used Llama 2 open source LLM using Gradient embedding and Astra DB (Apache Cassandra) for  efficiently responding to queries where the answers need to be derived from extensive PDF documents (exceeding 100 pages). ##



<a href="https://colab.research.google.com/github/bhattbhavesh91//pdf-q-a-llamaindex-llama2/blob/main/pdf-q-a-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [1]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Import OS & JSON Modules

In [2]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = userdata.get('GRADIENT_ACCESS_TOKEN')
os.environ['GRADIENT_WORKSPACE_ID'] = userdata.get('GRADIENT_WORKSPACE_ID')

In [3]:
!pip install llama-index-llms-gradient
!pip install llama-index-embeddings-gradient

# Import Cassandra & llama Index

In [4]:

from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM


# Connect to the VectorDB

# Define the Gradient's Model Adapter for LLAMA-2

In [5]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

# Configure Gradient embeddings

In [6]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [7]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

<ipython-input-7-944266470549>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


# Load the PDFs

Load the pdfs to Documents folder

In [9]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 109 document(s).


# Setup and Query Index

In [10]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

# **Responses for the pdf -"Blade Runner 2049"**

In [11]:
response = query_engine.query("Explain the theme of the movie?")
print(response)

 Based on the context provided, the theme of the movie "Blade Runner 2049" can be inferred as:

1. The blurring of lines between human and replicant: The movie explores the idea of what it means to be human, and how the distinction between humans and replicants is becoming increasingly blurred. This is evident in the character of K, who is a replicant but identifies as human, and Deckard, who is a blade runner tasked with hunting down replicants but struggles with his own humanity.
2. The search for identity and self-discovery: The movie delves into the themes of identity and self-discovery, as the characters grapple with their own existence and purpose. This is seen in the character of Luv, who is searching for her creator and seeking answers about her own identity, and Deckard, who is on a quest to uncover the truth about his own past and the nature of his existence.
3. The consequences of playing God: The movie explores the dangers of tampering with the natural order of things and t

In [12]:
response = query_engine.query("Who are the characters?")
print(response)

 Based on the context information provided, the characters in the passage are:

1. K: A mysterious figure who is manipulating the data and searching for attendant files.
2. Mariette: A woman who is revealed to be the same person as the woman in an old photo.
3. Freysa: A woman who fought with Sapper on Calantha and is the subject of a photo that Mariette shows to K.
4. Rachael: A woman who is mentioned in the passage as someone who was with Freysa.

It is important to note that the passage does not provide any direct evidence to confirm the identities of these characters, and their relationships with each other are not fully explained. However, based on the context and the information provided, these are the most likely interpretations of the characters in the passage.


In [13]:
response = query_engine.query("How many male and female characters are in the movie?")
print(response)

 Based on the context information provided, there are two characters in the movie:

1. Male character (labelled as MALE 4847)
2. Female character (labelled as FEMALE 2181, deceased)

Therefore, there is one male character and one female character in the movie.


In [14]:
response = query_engine.query("Does the script pass the Bechdel test?")
print(response)

 Based on the provided context, the script does not pass the Bechdel test. The only named female character in the script is "the girl," who dies early on in the story. The remaining characters are male, and there is no mention of any other female characters. Therefore, the script fails the Bechdel test.


In [15]:
response = query_engine.query("What is the role of Deckard in the movie?")
print(response)

 Based on the provided context, it can be inferred that Deckard is a bounty hunter or a cop who is tasked with capturing or killing a character named K. Deckard is shown to be skilled in hand-to-hand combat and weaponry, as he is able to fight off multiple attackers and disarm them. However, K proves to be a formidable opponent and is able to inflict significant damage on Deckard, including puncturing his abdomen with debris and breaking his hand. Despite this, Deckard continues to pursue K, indicating that he is determined to complete his mission. Overall, Deckard's role in the movie appears to be that of a skilled and determined bounty hunter or cop who is tasked with capturing or killing a dangerous criminal.
